# Navigation Project

---

## Project Description

This document is the report of the first Udacity project for reinforcment learning. 

For this project, I have train an agent to navigate and collect yellow bananas in a large, square world.

<img src="images/banana.gif">

A reward of +1 is provided for collecting a yellow banana, and a reward of -1 is provided for collecting a blue banana. Thus, the goal of the agent is to collect as many yellow bananas as possible while avoiding blue bananas.

The state space has 37 dimensions and contains the agent's velocity, along with ray-based perception of objects around the agent's forward direction. Given this information, the agent has to learn how to best select actions. Four discrete actions are available, corresponding to:

* 0 - move forward.
* 1 - move backward.
* 2 - turn left.
* 3 - turn right.

The task is episodic, and in order to solve the environment, the agent must get an average score of +13 over 100 consecutive episodes.

## Solution Approach

The solution follows the vanilla approach by [the original DQN paper](https://arxiv.org/abs/1312.5602).
But instead of having convolutional layers to derive the state from raw pixel data, we can directly access the crisp current state.
Therefore, the whole network architecture simplifies, such that I only need two deep layers with [ReLU activation](https://arxiv.org/pdf/1706.08098.pdf) for the Q-Network.
For further investigation, I evaluate the size of the hidden layers to find the best estimate following a fan-in approach for deeper layers.

The hyperparameter for training the Q-Network are as follows:

* DQN Learning Parameters
 * $n_{\text{episodes}}$ = 2000
 * $\epsilon_{\text{decay}}$ = 0.995
 * $\epsilon_{\text{start}}$ = 1.0
 * $\epsilon_{\text{end}}$ = 0.01
 * Replay buffer size = 10e5
 * Minibatch size = 64
 * $\Gamma$ (aka discount factor) = 0.99
 * $\tau$ (aka soft update of target parameters) = 1e-3
 * Learning rate = 5e-4
 * Update Interval between local and target network = 4
* DQN Network Architecture
 * Layer-Conections: Input -> ReLU Layer 1 -> RelU Layer 2 -> Output
 * Layer-Dimensionality [ReLU Layer 1, ReLU Layer 2]: [8,8], [16,8], [16,16], [32,8], [32,16], [32,32], [64,8], [64,16], [64,32], [64,64], [128,8], [128,16], [128,32], [128,64], [128,128]


## Evaluation

### 1. Preliminars for the Program Environment

In [ ]:
!pip -q install ./python

from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import pickle
from pathlib import Path
from functions.dqn_agent import Agent
from functions.rl import DQN
from functions.helper import env_info, scores_stat

# Start the environment
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

state_size, action_size = env_info(env, brain)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 1
Number of actions: 4
States look like: [ 1.          0.          0.          0.          0.84408134  0.          0.
  1.          0.          0.0748472   0.          1.          0.          0.
  0.25755     1.          0.          0.          0.          0.74177343
  0.          1.          0.          0.          0.25854847  0.          0.
  1.          0.          0.09355672  0.          1.          0.          0.
  0.31969345  0.          0.        ]
States have length: 37


### 2. Training the agent using a 2-Layer DQN with varying Layer-Sizes

In [ ]:
n_episodes = 2000
eps_decay = 0.995
fc_units_power_min = 3
fc_units_power_max = 6
scores_fc_eval = []
for fc_1_units_power in np.arange(fc_units_power_min, fc_units_power_max+1):
    for fc_2_units_power in np.arange(fc_units_power_min, fc_1_units_power+1):
        # Allocate the agent
        agent = Agent(state_size=state_size,
                      action_size=action_size,
                      seed=0,
                      fc1_units=np.int(2**fc_1_units_power),
                      fc2_units=np.int(2**fc_2_units_power))
        # Define a config string
        filename =  str("_fc1-" + str(agent.fc1_units) +
                    "_fc2-" + str(agent.fc2_units) +
                    "_e-" + str(n_episodes) +
                    "_decay-" + str(eps_decay))
        print("Current config: " + filename)
        # Check if the scores are arcived, otherwise train it
        if not Path("eval/dict" + filename).is_file():
            # Train the agent
            dqn = DQN(agent = agent, env = env, brain = brain,
                      eps_decay = eps_decay,
                      verbose = False,
                      n_episodes = n_episodes,
                      max_t = 1000)
            # Archive the values
            scores = dqn.train(weights_location = "weights/network" + filename + ".pth")
            scores_fc_eval.append({"fc1_units": agent.fc1_units,
                                   "fc2_units": agent.fc2_units,
                                   "epsilodes": dqn.n_episodes,
                                   "decay": dqn.eps_decay,
                                   "scores": scores})
            with open("eval/dict" + filename, 'wb') as fp:
                pickle.dump(scores_fc_eval[-1], fp)
        else:
            scores_fc_eval.append(pickle.load( open( "eval/dict" + filename, "rb" ) ))
env.close()

Current config: _fc1-8_fc2-8_e-2000_decay-0.995
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


## 3. Evaluation

In [ ]:
# Three subplots sharing both x/y axes
plot_num = fc_units_power_max - fc_units_power_min + 1
f, axs = plt.subplots(plot_num, plot_num, figsize=(20,10), sharex=True, sharey=True)
cnt = 0
threashold = 13
eval_threashold_idx = np.zeros((plot_num,plot_num))
eval_max_score = np.zeros((plot_num,plot_num))
for idx in np.arange(0,plot_num):
    for idy in np.arange(0,idx+1):
        ax = axs[idy, plot_num - idx - 1]
        if idy == 0:
            ax.set_title(str(scores_fc_eval[cnt]['fc1_units']))
        if idx == plot_num - 1:
            ax.set_ylabel(str(scores_fc_eval[cnt]['fc2_units']))
        scores = scores_fc_eval[cnt]['scores']
        scores_avg, scores_std = scores_stat(scores, 100)
        eval_max_score[idy, idx] = np.nanmax(scores_avg)
        eval_threashold_idx[idy, idx] = np.nanargmax(scores_avg > threashold)
        ax.plot(np.arange(len(scores)), scores, linestyle='-', color='cornflowerblue', linewidth=1)
        ax.plot(np.arange(len(scores)), scores_avg, linestyle='-', color='red', linewidth=2)
        ax.plot([0, len(scores)-1], [threashold, threashold], linestyle='--', color='black', linewidth=2)
        cnt = cnt + 1
f.subplots_adjust(hspace=0)
f.subplots_adjust(wspace=0)
f.suptitle("Comparision of layer dimensionality: FC1 (x-axis) vs. FC2 (y-axis)")

Corresponding index, where the average score exceeds 13 for the first time:

In [ ]:
print(eval_threashold_idx)

Corresponding maximum score achived

In [ ]:
print(eval_max_score)

### 4. Conclusion

The evaluation shows, that 